# Limpieza de Datos del Dataset IGAC

Este notebook procesa el dataset completo aplicando funciones de limpieza.

## Objetivos
1. Procesar el dataset completo en chunks
2. Limpiar nombres de municipios y departamentos
3. Validar y corregir fechas
4. Limpiar valores numéricos
5. Manejar valores nulos
6. Eliminar duplicados
7. Guardar datos limpios en formato Parquet

In [1]:
# Importar librerías
import sys
sys.path.append('../src')


import polars as pl
from pathlib import Path
from tqdm import tqdm
from etl.data_loader import load_full_dataset_lazy, save_to_parquet
from etl.data_cleaner import apply_all_cleaning
from utils.config import CLEANED_PARQUET_FILE

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 1. Carga del Dataset Completo (Lazy Loading)

In [2]:
# Cargar dataset en modo lazy (no se carga en memoria hasta ejecutar operaciones)
lf = load_full_dataset_lazy()
print("Dataset cargado en modo lazy")
print(f"Columnas: {lf.columns}")

Cargando dataset en modo lazy (sin cargar en memoria)...
✓ Dataset cargado en modo lazy
Dataset cargado en modo lazy
Columnas: ['PK', 'MATRICULA', 'FECHA_RADICA_TEXTO', 'FECHA_APERTURA_TEXTO', 'YEAR_RADICA', 'ORIP', 'DIVIPOLA', 'DEPARTAMENTO', 'MUNICIPIO', 'TIPO_PREDIO_ZONA', 'CATEGORIA_RURALIDAD', 'NUM_ANOTACION', 'ESTADO_FOLIO', 'FOLIOS_DERIVADOS', 'Dinámica_Inmobiliaria', 'COD_NATUJUR', 'NOMBRE_NATUJUR', 'NUMERO_CATASTRAL', 'NUMERO_CATASTRAL_ANTIGUO', 'DOCUMENTO_JUSTIFICATIVO', 'COUNT_A', 'COUNT_DE', 'PREDIOS_NUEVOS', 'TIENE_VALOR', 'TIENE_MAS_DE_UN_VALOR', 'VALOR']


C:\Users\Dylber Cabrera\AppData\Local\Temp\ipykernel_41228\4052329036.py:4: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  print(f"Columnas: {lf.columns}")


In [3]:
# Conteo de nulos por columna (compatible con pl.LazyFrame)
nulls_row = lf.select([pl.col(c).is_null().sum().alias(c) for c in lf.columns]).collect()
total_rows = lf.select(pl.count().alias('total')).collect()['total'][0]

cols = nulls_row.columns
null_df = pl.DataFrame({
    'Columna': cols,
    'Nulos': [nulls_row[c][0] for c in cols],
    'Porcentaje': [(nulls_row[c][0] / total_rows) * 100 if total_rows else 0 for c in cols]
}).filter(pl.col('Nulos') > 0).sort('Nulos', descending=True)

# Mostrar en Jupyter como tabla más legible
display(null_df.to_pandas())

C:\Users\Dylber Cabrera\AppData\Local\Temp\ipykernel_41228\3336887280.py:2: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  nulls_row = lf.select([pl.col(c).is_null().sum().alias(c) for c in lf.columns]).collect()
C:\Users\Dylber Cabrera\AppData\Local\Temp\ipykernel_41228\3336887280.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows = lf.select(pl.count().alias('total')).collect()['total'][0]


,Columna,Nulos,Porcentaje
0,NUMERO_CATASTRAL,30903248,100.000000
1,NUMERO_CATASTRAL_ANTIGUO,30903248,100.000000
2,FOLIOS_DERIVADOS,30601558,99.023760
3,FECHA_APERTURA_TEXTO,14369327,46.497789
4,ESTADO_FOLIO,13579939,43.943404
5,VALOR,12683596,41.042922
6,ORIP,5265699,17.039306
7,COD_NATUJUR,89084,0.288267
8,NUM_ANOTACION,26672,0.086308
9,DOCUMENTO_JUSTIFICATIVO,7748,0.025072


## 2. Aplicar Limpieza

Aplicamos todas las funciones de limpieza definidas en el módulo `data_cleaner`.

In [4]:
#eliminamos comillas dobles de todas las columnas de tipo string
string_cols = [
    col for col, dtype in lf.collect_schema().items()
    if dtype == pl.Utf8
]
string_cols

lf = lf.with_columns([
    pl.col(col).str.replace('"', '') for col in string_cols
])



In [5]:
sample = lf.limit(5000).collect()

string_cols = [
    col for col, dtype in sample.schema.items()
    if dtype == pl.Utf8
]

# Revisamos solo 5000 filas, no 30 millones
check_sample = sample.select([
    pl.col(col).str.contains('"').any().alias(col)
    for col in string_cols
])

check_sample


PK,MATRICULA,FECHA_RADICA_TEXTO,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,ESTADO_FOLIO,FOLIOS_DERIVADOS,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,VALOR
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
false,false,false,false,false,false,false,false,false,false,false,false,false,false


In [6]:
lf.head().collect()


PK,MATRICULA,FECHA_RADICA_TEXTO,FECHA_APERTURA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,datetime[μs],i64,i64,i64,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,str
"""05001-001-000013-00014-00313-2…","""001-13""","""2018-02-05 00:00:00""",1973-01-11 00:00:00,2018,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",14,null,null,0,313,"""CONSTITUCIÓN DE FIDEICOMISO CI…",null,null,"""01-ESCRITURA 3321 DE 22-12-201…",2,1,0,0,0,null
"""05001-001-000014-00015-00313-2…","""001-14""","""2018-02-05 00:00:00""",1973-01-11 00:00:00,2018,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",15,null,null,0,313,"""CONSTITUCIÓN DE FIDEICOMISO CI…",null,null,"""01-ESCRITURA 3321 DE 22-12-201…",2,1,0,0,0,null
"""05001-001-000023-00007-00109-2…","""001-23""","""2020-12-02 00:00:00""",1973-01-10 00:00:00,2020,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",7,null,null,1,109,"""ADJUDICACIÓN EN SUCESION""",null,null,"""01-ESCRITURA 3322 DE 30-10-202…",5,1,0,0,0,null
"""05001-001-000065-00013-00317-2…","""001-65""","""2015-12-16 00:00:00""",1973-01-10 00:00:00,2015,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",13,null,null,1,317,"""CONSTITUCIÓN REGLAMENTO DE PRO…",null,null,"""01-ESCRITURA 8145 DE 04-12-201…",0,1,0,0,0,null
"""05001-001-000067-00014-00109-2…","""001-67""","""2016-03-29 00:00:00""",1973-01-11 00:00:00,2016,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",14,null,null,1,109,"""ADJUDICACIÓN EN SUCESION""",null,null,"""01-ESCRITURA 1266 DE 10-03-201…",1,1,0,0,0,null


In [7]:
# Eliminamos la columna de FECHA_APERTURA_TEXTO ya que no es relevante
lf = lf.drop("FECHA_APERTURA_TEXTO")

In [8]:
lf.head().collect()

PK,MATRICULA,FECHA_RADICA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,i64,i64,i64,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,str
"""05001-001-000013-00014-00313-2…","""001-13""","""2018-02-05 00:00:00""",2018,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",14,null,null,0,313,"""CONSTITUCIÓN DE FIDEICOMISO CI…",null,null,"""01-ESCRITURA 3321 DE 22-12-201…",2,1,0,0,0,null
"""05001-001-000014-00015-00313-2…","""001-14""","""2018-02-05 00:00:00""",2018,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",15,null,null,0,313,"""CONSTITUCIÓN DE FIDEICOMISO CI…",null,null,"""01-ESCRITURA 3321 DE 22-12-201…",2,1,0,0,0,null
"""05001-001-000023-00007-00109-2…","""001-23""","""2020-12-02 00:00:00""",2020,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",7,null,null,1,109,"""ADJUDICACIÓN EN SUCESION""",null,null,"""01-ESCRITURA 3322 DE 30-10-202…",5,1,0,0,0,null
"""05001-001-000065-00013-00317-2…","""001-65""","""2015-12-16 00:00:00""",2015,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",13,null,null,1,317,"""CONSTITUCIÓN REGLAMENTO DE PRO…",null,null,"""01-ESCRITURA 8145 DE 04-12-201…",0,1,0,0,0,null
"""05001-001-000067-00014-00109-2…","""001-67""","""2016-03-29 00:00:00""",2016,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",14,null,null,1,109,"""ADJUDICACIÓN EN SUCESION""",null,null,"""01-ESCRITURA 1266 DE 10-03-201…",1,1,0,0,0,null


In [ ]:
resultado = (
    lf
    .filter(pl.col("MATRICULA") == "300-338299")
    .select("FECHA_RADICA_TEXTO")  
)

print(resultado)


shape: (4, 1)
┌─────────────────────┐
│ FECHA_RADICA_TEXTO  │
│ ---                 │
│ str                 │
╞═════════════════════╡
│ 2018-01-15 00:00:00 │
│ 2018-01-15 00:00:00 │
│ 24/01/22            │
│ 24/01/22            │
└─────────────────────┘


In [10]:

lf.select([
    pl.col("DOCUMENTO_JUSTIFICATIVO").null_count().alias("nulos")
]).collect()

nulos
u32
7748


In [11]:
lf.head().collect()

PK,MATRICULA,FECHA_RADICA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,i64,i64,i64,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,str
"""05001-001-000013-00014-00313-2…","""001-13""","""2018-02-05 00:00:00""",2018,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",14,null,null,0,313,"""CONSTITUCIÓN DE FIDEICOMISO CI…",null,null,"""01-ESCRITURA 3321 DE 22-12-201…",2,1,0,0,0,null
"""05001-001-000014-00015-00313-2…","""001-14""","""2018-02-05 00:00:00""",2018,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",15,null,null,0,313,"""CONSTITUCIÓN DE FIDEICOMISO CI…",null,null,"""01-ESCRITURA 3321 DE 22-12-201…",2,1,0,0,0,null
"""05001-001-000023-00007-00109-2…","""001-23""","""2020-12-02 00:00:00""",2020,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",7,null,null,1,109,"""ADJUDICACIÓN EN SUCESION""",null,null,"""01-ESCRITURA 3322 DE 30-10-202…",5,1,0,0,0,null
"""05001-001-000065-00013-00317-2…","""001-65""","""2015-12-16 00:00:00""",2015,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",13,null,null,1,317,"""CONSTITUCIÓN REGLAMENTO DE PRO…",null,null,"""01-ESCRITURA 8145 DE 04-12-201…",0,1,0,0,0,null
"""05001-001-000067-00014-00109-2…","""001-67""","""2016-03-29 00:00:00""",2016,1,5001,"""ANTIOQUIA""","""MEDELLÍN""","""URBANO""","""Ciudades y aglomeraciones""",14,null,null,1,109,"""ADJUDICACIÓN EN SUCESION""",null,null,"""01-ESCRITURA 1266 DE 10-03-201…",1,1,0,0,0,null


In [12]:
# Recolectar el LazyFrame a DataFrame y aplicar limpieza
print("Ejecutando limpieza del dataset completo...")
print("ADVERTENCIA: Este proceso puede tomar varios minutos debido al tamaño del dataset.\n")

# Collect lazy frame
df = lf.collect()
print(f"Dataset cargado en memoria: {df.shape[0]:,} filas")
    
# Aplicar limpieza
df_cleaned = apply_all_cleaning(df)
print(f"\nDataset limpio: {df_cleaned.shape[0]:,} filas")


Ejecutando limpieza del dataset completo...
ADVERTENCIA: Este proceso puede tomar varios minutos debido al tamaño del dataset.

Dataset cargado en memoria: 30,903,248 filas

🚀 INICIANDO PROCESO COMPLETO DE LIMPIEZA
🧹 LIMPIANDO COLUMNAS STRING...
   ➤ Columnas detectadas: ['MATRICULA', 'DIVIPOLA', 'TIPO_PREDIO_ZONA', 'CATEGORIA_RURALIDAD', 'ESTADO_FOLIO', 'NOMBRE_NATUJUR']
✓ Columnas string limpiadas
→ Normalizando nombres de municipios...
Normalizando nombres de departamentos...
✓ Departamentos normalizados
  📅 Parseando y estandarizando fechas...
     Procesando FECHA_RADICA_TEXTO (tipo: String)...
       ⚠ 3,370,823 fechas no parseadas
  ✓ Fechas estandarizadas al formato dd/mm/yyyy
Limpiando valores numéricos...
✓ Valores numéricos limpios
Manejando valores nulos: estrategia 'fill'
     NUMERO_CATASTRAL: 30,903,248 nulos detectados
     NUMERO_CATASTRAL_ANTIGUO: 30,903,248 nulos detectados
     ESTADO_FOLIO: 13,579,939 nulos detectados
     FOLIOS_DERIVADOS: 30,601,558 nulos detecta

In [13]:
# Contar valores nulos por columna
null_counts = df_cleaned.null_count()
null_df = pl.DataFrame({
    'Columna': df_cleaned.columns,
    'Nulos': [null_counts[col][0] for col in df_cleaned.columns],
    'Porcentaje': [(null_counts[col][0] / len(df_cleaned)) * 100 for col in df_cleaned.columns]
})

null_df = null_df.filter(pl.col('Nulos') > 0).sort('Nulos', descending=True)
print("\nColumnas con valores nulos:")
print(null_df)


Columnas con valores nulos:
shape: (1, 3)
┌────────────────────┬─────────┬────────────┐
│ Columna            ┆ Nulos   ┆ Porcentaje │
│ ---                ┆ ---     ┆ ---        │
│ str                ┆ i64     ┆ f64        │
╞════════════════════╪═════════╪════════════╡
│ FECHA_RADICA_TEXTO ┆ 3370824 ┆ 10.907669  │
└────────────────────┴─────────┴────────────┘


In [14]:
df_cleaned.head()

PK,MATRICULA,FECHA_RADICA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,f64,i64,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""25290-157-075628-00023-00125-2…","""157-75628""","""30/07/2018""",2018.0,157,"""25290""","""CUNDINAMARCA""","""FUSAGASUGA""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",23,"""INACTIVO""","""SIN_FOLIO""",1,125,"""COMPRAVENTA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""01-ESCRITURA 1862 DE 17-07-201…",1.0,1.0,0.0,0.0,0.0,0.0
"""25489-162-040332-00001-00350-2…","""162-40332""","""22/11/2021""",2021.0,162,"""25489""","""CUNDINAMARCA""","""NIMAIMA""","""RURAL""","""RURAL""",1,"""ACTIVO""","""SIN_FOLIO""",0,350,"""SERVIDUMBRE DE AGUA ACTIVA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-ESCRITURA 1406 DE 24/09/202…",0.0,3.0,1.0,1.0,0.0,0.0
"""13244-062-027309-00008-00907-2…","""062-27309""",null,2022.0,62,"""13244""","""BOLÍVAR""","""EL CARMEN DE BOLIVAR""","""URBANO""","""INTERMEDIO""",8,"""ACTIVO""","""SIN_FOLIO""",0,907,"""CAMBIO DE RAZÓN SOCIAL""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-ESCRITURA 186 DEL 22/04/22;…",1.0,1.0,0.0,1.0,0.0,0.0
"""68385-324-051688-00011-00125-2…","""324-51688""","""27/12/2018""",2018.0,324,"""68385""","""SANTANDER""","""LANDAZURI""","""RURAL""","""RURAL""",11,"""INACTIVO""","""SIN_FOLIO""",1,125,"""COMPRAVENTA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""01-ESCRITURA""",1.0,1.0,0.0,0.0,0.0,0.0
"""63130-282-041924-00004-00186-2…","""282-41924""","""31/05/2017""",2017.0,282,"""63130""","""QUINDÍO""","""CALARCA""","""RURAL""","""CIUDADES Y AGLOMERACIONES""",4,"""INACTIVO""","""SIN_FOLIO""",1,186,"""TRANSFERENCIA A TÍTULO DE SUBS…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""16-RESOLUCION 358 DE 02-05-201…",1.0,1.0,0.0,0.0,0.0,0.0


In [15]:
# convierto a string el campo ORIP
if "ORIP" in df_cleaned.columns:
    df_cleaned = df_cleaned.with_columns(
        pl.col("ORIP")
          .cast(pl.Utf8, strict=False)        # asegurar tipo string
          .str.strip_chars()                   # quitar espacios / caracteres invisibles
          .str.to_uppercase()                  # pasar a mayúsculas
          .alias("ORIP")
    )
    
# reemplazo nulos, cadenas vacías y "0"/"0.0" por "SIN_ORIP"
if "ORIP" in df_cleaned.columns:
    df_cleaned = df_cleaned.with_columns(
        pl.when(
            pl.col("ORIP").is_null()
            | (pl.col("ORIP").cast(pl.Utf8, strict=False).str.strip_chars() == "")
            | (pl.col("ORIP").cast(pl.Utf8, strict=False).str.strip_chars().is_in(["0", "0.0"]))
        )
        .then(pl.lit("SIN_ORIP"))
        .otherwise(
            pl.col("ORIP").cast(pl.Utf8, strict=False)
                       .str.strip_chars()
                       .str.to_uppercase()
        )
        .alias("ORIP")
    )

In [16]:
df_cleaned.head()

PK,MATRICULA,FECHA_RADICA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,f64,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""05615-020-048347-00006-00842-2…","""020-48347""","""08/02/2022""",2022.0,"""20""","""5615""","""ANTIOQUIA""","""RIONEGRO""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",6,"""INACTIVO""","""SIN_FOLIO""",0,842,"""CANCELACIÓN PROVIDENCIA ADMINI…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""04-OFICIO 1050-194 DE 02-02-20…",0.0,1.0,0.0,0.0,0.0,0.0
"""05585-019-014037-00005-00427-2…","""019-14037""","""26/01/2017""",2017.0,"""19""","""5585""","""ANTIOQUIA""","""PUERTO NARE""","""URBANO""","""RURAL""",5,"""ACTIVO""","""SIN_FOLIO""",0,427,"""EMBARGO EJECUTIVO CON ACCIÓN P…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-OFICIO 1233 DE 13/12/2016; …",1.0,1.0,0.0,1.0,0.0,0.0
"""11001-50C-2037962-00004-00125-…","""50C-2037962""","""14/01/2019""",2019.0,"""SIN_ORIP""","""11001""","""BOGOTÁ D.C.""","""BOGOTÁ""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",4,"""INACTIVO""","""SIN_FOLIO""",1,125,"""COMPRAVENTA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""01-ESCRITURA 4128 DE 17-12-201…",1.0,1.0,0.0,0.0,0.0,0.0
"""25386-166-060114-00007-00900-2…","""166-60114""","""30/07/2015""",2015.0,"""166""","""25386""","""CUNDINAMARCA""","""LA MESA""","""RURAL""","""INTERMEDIO""",7,"""ACTIVO""","""SIN_FOLIO""",0,900,"""OTRO / OFERTA DE COMPRA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-OFICIO 201552122533 DE 22/0…",0.0,1.0,0.0,1.0,0.0,0.0
"""47245-224-021752-00003-00186-2…","""224-21752""","""22/07/0022""",2022.0,"""224""","""47245""","""MAGDALENA""","""EL BANCO""","""URBANO""","""INTERMEDIO""",3,"""ACTIVO""","""SIN_FOLIO""",1,186,"""TRANSFERENCIA A TÍTULO DE SUBS…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-ESCRITURA 2087 DEL 01/07/22…",1.0,1.0,0.0,1.0,0.0,0.0


In [17]:
null_counts = df_cleaned.null_count()
null_df = pl.DataFrame({
    'Columna': df_cleaned.columns,
    'Nulos': [null_counts[col][0] for col in df_cleaned.columns],
    'Porcentaje': [(null_counts[col][0] / len(df_cleaned)) * 100 for col in df_cleaned.columns]
})

null_df = null_df.filter(pl.col('Nulos') > 0).sort('Nulos', descending=True)
print("\nColumnas con valores nulos:")
print(null_df)


Columnas con valores nulos:
shape: (1, 3)
┌────────────────────┬───────┬────────────┐
│ Columna            ┆ Nulos ┆ Porcentaje │
│ ---                ┆ ---   ┆ ---        │
│ str                ┆ i64   ┆ f64        │
╞════════════════════╪═══════╪════════════╡
│ FECHA_RADICA_TEXTO ┆ 4     ┆ 0.000013   │
└────────────────────┴───────┴────────────┘


In [18]:
# Eliminar filas donde FECHA_RADICA_TEXTO es null o cadena vacía
if "FECHA_RADICA_TEXTO" in df_cleaned.columns:
    df_cleaned = df_cleaned.filter(
        ~(
            pl.col("FECHA_RADICA_TEXTO").is_null()
            | (pl.col("FECHA_RADICA_TEXTO").cast(pl.Utf8, strict=False).str.strip_chars() == "")
        )
    )

In [19]:
null_counts = df_cleaned.null_count()
null_df = pl.DataFrame({
    'Columna': df_cleaned.columns,
    'Nulos': [null_counts[col][0] for col in df_cleaned.columns],
    'Porcentaje': [(null_counts[col][0] / len(df_cleaned)) * 100 for col in df_cleaned.columns]
})

null_df = null_df.filter(pl.col('Nulos') > 0).sort('Nulos', descending=True)
print("\nColumnas con valores nulos:")
print(null_df)


Columnas con valores nulos:
shape: (0, 3)
┌─────────┬───────┬────────────┐
│ Columna ┆ Nulos ┆ Porcentaje │
│ ---     ┆ ---   ┆ ---        │
│ str     ┆ i64   ┆ f64        │
╞═════════╪═══════╪════════════╡
└─────────┴───────┴────────────┘


## 3. Verificar Resultados de Limpieza

In [20]:
# Comparar antes y después
print("\n" + "="*60)
print("COMPARACIÓN ANTES/DESPUÉS DE LIMPIEZA")
print("="*60)
print(f"\nFilas originales: {df.shape[0]:,}")
print(f"Filas después de limpieza: {df_cleaned.shape[0]:,}")
print(f"Filas eliminadas: {df.shape[0] - df_cleaned.shape[0]:,}")
print(f"Porcentaje retenido: {(df_cleaned.shape[0] / df.shape[0]) * 100:.2f}%")


COMPARACIÓN ANTES/DESPUÉS DE LIMPIEZA

Filas originales: 30,903,248
Filas después de limpieza: 30,903,244
Filas eliminadas: 4
Porcentaje retenido: 100.00%


In [21]:
# Ver muestra de datos limpios
print("\nMuestra de datos limpios:")
df_cleaned.head(10)


Muestra de datos limpios:


PK,MATRICULA,FECHA_RADICA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,f64,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""05615-020-048347-00006-00842-2…","""020-48347""","""08/02/2022""",2022.0,"""20""","""5615""","""ANTIOQUIA""","""RIONEGRO""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",6,"""INACTIVO""","""SIN_FOLIO""",0,842,"""CANCELACIÓN PROVIDENCIA ADMINI…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""04-OFICIO 1050-194 DE 02-02-20…",0.0,1.0,0.0,0.0,0.0,0.0
"""05585-019-014037-00005-00427-2…","""019-14037""","""26/01/2017""",2017.0,"""19""","""5585""","""ANTIOQUIA""","""PUERTO NARE""","""URBANO""","""RURAL""",5,"""ACTIVO""","""SIN_FOLIO""",0,427,"""EMBARGO EJECUTIVO CON ACCIÓN P…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-OFICIO 1233 DE 13/12/2016; …",1.0,1.0,0.0,1.0,0.0,0.0
"""11001-50C-2037962-00004-00125-…","""50C-2037962""","""14/01/2019""",2019.0,"""SIN_ORIP""","""11001""","""BOGOTÁ D.C.""","""BOGOTÁ""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",4,"""INACTIVO""","""SIN_FOLIO""",1,125,"""COMPRAVENTA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""01-ESCRITURA 4128 DE 17-12-201…",1.0,1.0,0.0,0.0,0.0,0.0
"""25386-166-060114-00007-00900-2…","""166-60114""","""30/07/2015""",2015.0,"""166""","""25386""","""CUNDINAMARCA""","""LA MESA""","""RURAL""","""INTERMEDIO""",7,"""ACTIVO""","""SIN_FOLIO""",0,900,"""OTRO / OFERTA DE COMPRA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-OFICIO 201552122533 DE 22/0…",0.0,1.0,0.0,1.0,0.0,0.0
"""47245-224-021752-00003-00186-2…","""224-21752""","""22/07/0022""",2022.0,"""224""","""47245""","""MAGDALENA""","""EL BANCO""","""URBANO""","""INTERMEDIO""",3,"""ACTIVO""","""SIN_FOLIO""",1,186,"""TRANSFERENCIA A TÍTULO DE SUBS…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-ESCRITURA 2087 DEL 01/07/22…",1.0,1.0,0.0,1.0,0.0,0.0
"""05360-001-1382264-00010-00901-…","""001-1382264""","""05/04/2022""",2022.0,"""1""","""5360""","""ANTIOQUIA""","""ITAGUI""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",10,"""INACTIVO""","""SIN_FOLIO""",0,901,"""ACLARACIÓN""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""01-ESCRITURA 1206 DE 23-02-202…",0.0,1.0,0.0,0.0,0.0,0.0
"""25873-154-051403-00001-01012-2…","""154-51403""","""24/06/2022""",2022.0,"""154""","""25873""","""CUNDINAMARCA""","""VILLAPINZON""","""RURAL""","""INTERMEDIO""",1,"""INACTIVO""","""SIN_FOLIO""",0,0,"""DECLARACION DE TITULACION DE L…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""16-RESOLUCION 00093376 DE 29-0…",2.0,1.0,1.0,0.0,0.0,0.0
"""76001-370-1033323-00003-00843-…","""370-1033323""","""17/09/2021""",2021.0,"""370""","""76001""","""VALLE DEL CAUCA""","""SANTIAGO DE CALI""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",3,"""INACTIVO""","""SIN_FOLIO""",0,843,"""CANCELACIÓN POR VOLUNTAD DE LA…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""01-ESCRITURA 1169 DE 30-07-202…",1.0,1.0,0.0,1.0,0.0,0.0
"""76275-378-241826-00003-00458-2…","""378-241826""","""07/09/0021""",2021.0,"""378""","""76275""","""VALLE DEL CAUCA""","""FLORIDA""","""RURAL""","""CIUDADES Y AGLOMERACIONES""",3,"""ACTIVO""","""SIN_FOLIO""",0,458,"""PROHIBICIÓN ADMINISTRATIVA""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-RESOLUCION ADMINISTRATIVA 1…",2.0,1.0,0.0,1.0,0.0,0.0


In [22]:
# Verificar valores nulos en columnas críticas
critical_cols = ['PK', 'MUNICIPIO', 'DEPARTAMENTO', 'YEAR_RADICA', 'VALOR']

print("\nValores nulos en columnas críticas:")
for col in critical_cols:
    if col in df_cleaned.columns:
        null_count = df_cleaned[col].null_count()
        print(f"  {col}: {null_count}")


Valores nulos en columnas críticas:
  PK: 0
  MUNICIPIO: 0
  DEPARTAMENTO: 0
  YEAR_RADICA: 0
  VALOR: 0


## 4. Guardar Datos Limpios

In [23]:
# Guardar en formato Parquet (comprimido y eficiente)
save_to_parquet(df_cleaned, CLEANED_PARQUET_FILE, compression='snappy')
print(f"\n✓ Datos limpios guardados en: {CLEANED_PARQUET_FILE}")

Guardando datos en formato Parquet: d:\conscursoOpendata\igac-anomalias-inmobiliarias\notebooks\..\data\processed\igac_cleaned.parquet
✓ Archivo guardado: 2048.53 MB

✓ Datos limpios guardados en: d:\conscursoOpendata\igac-anomalias-inmobiliarias\notebooks\..\data\processed\igac_cleaned.parquet


## 5. Estadísticas Finales

In [24]:
# Estadísticas del dataset limpio
print("\nEstadísticas del dataset limpio:")
df_cleaned.describe()


Estadísticas del dataset limpio:


statistic,PK,MATRICULA,FECHA_RADICA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,str,f64,str,str,str,str,str,str,f64,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""30903244""","""30903244""","""30903244""",3.0903244e7,"""30903244""","""30903244""","""30903244""","""30903244""","""30903244""","""30903244""",3.0903244e7,"""30903244""","""30903244""",3.0903244e7,3.0903244e7,"""30903244""","""30903244""","""30903244""","""30903244""",3.0903244e7,3.0903244e7,3.0903244e7,3.0903244e7,3.0903244e7,3.0903244e7
"""null_count""","""0""","""0""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""",0.0,"""0""","""0""",0.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,2019.19465,null,null,null,null,null,null,8.245211,null,null,0.563075,451.10378,null,null,null,null,1.223572,1.009665,0.094102,0.589571,0.000033,0.02746
"""std""",null,null,null,2.651883,null,null,null,null,null,null,22.798172,null,null,0.496006,313.23865,null,null,null,null,2.315214,1.643787,0.291971,0.491912,0.00577,3.914648
"""min""","""05001-001-000013-00014-00313-2…","""001-1000000""","""01/01/0022""",2015.0,"""1""","""11001""","""AMAZONAS""","""ABEJORRAL""","""RURAL""","""CIUDADES Y AGLOMERACIONES""",0.0,"""ACTIVO""","""002-12624""",0.0,0.0,"""ABSTENERSE DE INSCRIBIR NUEVOS…","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""00-ACTA # 1 DE 22/11/2017; C…",0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",null,null,null,2017.0,null,null,null,null,null,null,3.0,null,null,0.0,186.0,null,null,null,null,1.0,1.0,0.0,0.0,0.0,0.0
"""50%""",null,null,null,2019.0,null,null,null,null,null,null,6.0,null,null,1.0,317.0,null,null,null,null,1.0,1.0,0.0,1.0,0.0,0.0
"""75%""",null,null,null,2022.0,null,null,null,null,null,null,9.0,null,null,1.0,843.0,null,null,null,null,1.0,1.0,0.0,1.0,0.0,0.0
"""max""","""99773-540-013896-00003-00311-2…","""999-9878""","""31/12/2022""",2023.0,"""SIN_ORIP""","""99773""","""VICHADA""","""ZONA BANANERA""","""URBANO""","""RURAL DISPERSO""",999.0,"""INACTIVO""","""SIN_FOLIO""",1.0,999.0,"""VALORIZACIÓN""","""SIN_NUMERO_CATASTRAL""","""SIN_NUMERO_CATASTRAL_ANTIGUO""","""SIN_DOCUMENTO_JUSTIFICATIVO""",994.0,609.0,1.0,1.0,1.0,999.12


In [25]:
# Distribución por año después de limpieza
year_dist = df_cleaned.group_by('YEAR_RADICA').agg([
    pl.count().alias('Cantidad')
]).sort('YEAR_RADICA')

print("\nDistribución por año (datos limpios):")
print(year_dist)

C:\Users\Dylber Cabrera\AppData\Local\Temp\ipykernel_36644\2993156178.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('Cantidad')



Distribución por año (datos limpios):
shape: (9, 2)
┌─────────────┬──────────┐
│ YEAR_RADICA ┆ Cantidad │
│ ---         ┆ ---      │
│ f64         ┆ u32      │
╞═════════════╪══════════╡
│ 2015.0      ┆ 3138029  │
│ 2016.0      ┆ 3656455  │
│ 2017.0      ┆ 3184604  │
│ 2018.0      ┆ 3020947  │
│ 2019.0      ┆ 3186849  │
│ 2020.0      ┆ 2671398  │
│ 2021.0      ┆ 3850367  │
│ 2022.0      ┆ 4223562  │
│ 2023.0      ┆ 3971033  │
└─────────────┴──────────┘


## Conclusiones

- Dataset limpio guardado en formato Parquet
- Valores nulos manejados
- Duplicados eliminados
- Nombres de municipios y departamentos normalizados
- Fechas parseadas correctamente
- Valores numéricos validados

**Próximo paso:** Notebook 03 - Estandarización de datos